# Interactive Demo: Fuzzy-Fairness DSS for LEO Satellites

This notebook demonstrates the complete end-to-end simulation and analysis pipeline.

## Phase 4: Complete Simulation + Visualization

## 1. Run Simulation

Execute a 30-second simulation with fuzzy policy.

In [ ]:
!python -m src.main --scenario urban_congestion_phase4 --policy fuzzy --gpu-id cpu --duration-s 30

## 2. Load Results

Load the CSV output from the simulation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set style
try:
    plt.style.use('seaborn-v0_8-darkgrid')
except:
    plt.style.use('seaborn-darkgrid')
sns.set_palette("husl")

# Load results
scenario_name = 'urban_congestion_phase4'
results_dir = 'results'
csv_path = f'{results_dir}/{scenario_name}_fuzzy.csv'

df = pd.read_csv(csv_path)
print(f"✓ Loaded {len(df)} time slots")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
df.head()

## 3. Fairness Metrics Over Time

Plot Jain Index and Fuzzy Fairness over time.

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Jain Index over time
axes[0].plot(df['slot'], df['jain'], label='Jain Index', linewidth=2, color='#2ecc71')
axes[0].set_xlabel('Time Slot', fontsize=12)
axes[0].set_ylabel('Jain Index', fontsize=12)
axes[0].set_title('Jain Fairness Index Over Time', fontsize=14, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Fuzzy Fairness over time
axes[1].plot(df['slot'], df['fuzzy_fairness'], label='Fuzzy Fairness', linewidth=2, color='#e74c3c')
axes[1].set_xlabel('Time Slot', fontsize=12)
axes[1].set_ylabel('Fuzzy Fairness', fontsize=12)
axes[1].set_title('Fuzzy Fairness Over Time', fontsize=14, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 1])

plt.tight_layout()
plt.show()

# Statistics
print(f"\nStatistics:")
print(f"  Mean Jain Index: {df['jain'].mean():.3f}")
print(f"  Mean Fuzzy Fairness: {df['fuzzy_fairness'].mean():.3f}")
print(f"  Std Jain Index: {df['jain'].std():.3f}")
print(f"  Std Fuzzy Fairness: {df['fuzzy_fairness'].std():.3f}")

## 4. User-Level Metrics (Per-Beam Fairness)

Analyze fairness distribution across different operators/beams.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Operator imbalance over time
axes[0].plot(df['slot'], df['operator_imbalance'], linewidth=2, color='#3498db')
axes[0].set_xlabel('Time Slot', fontsize=12)
axes[0].set_ylabel('Operator Imbalance (Gini)', fontsize=12)
axes[0].set_title('Operator Imbalance Over Time', fontsize=14, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_ylim([0, 1])

# Distribution of fairness scores
axes[1].hist(df['fuzzy_fairness'], bins=30, alpha=0.7, color='#9b59b6', edgecolor='black')
axes[1].set_xlabel('Fuzzy Fairness', fontsize=12)
axes[1].set_ylabel('Frequency', fontsize=12)
axes[1].set_title('Distribution of Fuzzy Fairness Scores', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')
axes[1].axvline(df['fuzzy_fairness'].mean(), color='red', linestyle='--', 
                label=f'Mean: {df["fuzzy_fairness"].mean():.3f}')
axes[1].legend()

plt.tight_layout()
plt.show()

print(f"\nOperator Imbalance Statistics:")
print(f"  Mean: {df['operator_imbalance'].mean():.3f}")
print(f"  Std: {df['operator_imbalance'].std():.3f}")
print(f"  Min: {df['operator_imbalance'].min():.3f}")
print(f"  Max: {df['operator_imbalance'].max():.3f}")

## 5. Map Scatter: Elevation vs Fairness

Visualize relationship between elevation angle and fairness (proxy using mean rate).

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))

# Use mean_rate as proxy for elevation effect
# Higher rate typically means better elevation
elevation_proxy = df['mean_rate'] / df['mean_rate'].max()  # Normalized
fairness = df['fuzzy_fairness']

scatter = ax.scatter(elevation_proxy, fairness, c=df['slot'], 
                    cmap='viridis', alpha=0.6, s=50, edgecolors='black', linewidth=0.5)

ax.set_xlabel('Elevation Effect (Normalized Rate)', fontsize=12)
ax.set_ylabel('Fuzzy Fairness', fontsize=12)
ax.set_title('Elevation vs Fairness', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Time Slot', fontsize=10)

plt.tight_layout()
plt.show()

# Correlation
correlation = np.corrcoef(elevation_proxy, fairness)[0, 1]
print(f"\nCorrelation (Elevation Proxy vs Fairness): {correlation:.3f}")

## 6. Interactive FIS Inference

Demonstrate fuzzy inference system with sample context and membership function visualization.

In [ ]:
from src.fairness.fuzzy_core import FuzzyInferenceSystem

# Initialize FIS
fis = FuzzyInferenceSystem(use_phase3=True)

# Sample context (normalized 0-1)
ctx_sample = {
    'throughput': 0.8,   # High throughput
    'latency': 0.2,      # Good latency (low)
    'outage': 0.1,       # Rare outage
    'priority': 0.9,    # High priority
    'doppler': 0.3,      # Low doppler
    'elevation': 0.85,   # High elevation
    'beam_load': 0.4     # Light load
}

# Compute fairness
fairness_for_user = fis.infer(ctx_sample)

print(f"Sample Context:")
for key, value in ctx_sample.items():
    print(f"  {key}: {value:.2f}")

print(f"\n✓ Computed Fuzzy Fairness: {fairness_for_user:.3f}")

# Visualize membership functions
fig, axes = plt.subplots(2, 4, figsize=(16, 8))
axes = axes.flatten()

input_names = ['throughput', 'latency', 'outage', 'priority', 
               'doppler', 'elevation', 'beam_load']

for idx, input_name in enumerate(input_names):
    if idx < len(axes):
        value = ctx_sample[input_name]
        
        # Get membership functions for this input
        if hasattr(fis, 'membership_sets') and fis.membership_sets:
            mfs = fis.membership_sets.get(input_name)
            if mfs:
                x = np.linspace(0, 1, 100)
                for label, mf_func in mfs.functions.items():
                    y = [mf_func(xi) for xi in x]
                    axes[idx].plot(x, y, label=label, linewidth=2)
                
                # Mark input value
                axes[idx].axvline(value, color='red', linestyle='--', 
                                label=f'Input: {value:.2f}')
                axes[idx].set_title(input_name.capitalize(), fontweight='bold')
                axes[idx].set_xlabel('Normalized Value')
                axes[idx].set_ylabel('Membership')
                axes[idx].legend(fontsize=8)
                axes[idx].grid(True, alpha=0.3)

# Output membership (fairness)
if len(input_names) < len(axes):
    output_mfs = fis.output_mfs
    x = np.linspace(0, 1, 100)
    for label, mf_func in output_mfs.functions.items():
        y = [mf_func(xi) for xi in x]
        axes[-1].plot(x, y, label=label, linewidth=2)
    
    axes[-1].axvline(fairness_for_user, color='red', linestyle='--', 
                    label=f'Output: {fairness_for_user:.3f}')
    axes[-1].set_title('Fairness Output', fontweight='bold')
    axes[-1].set_xlabel('Fairness Value')
    axes[-1].set_ylabel('Membership')
    axes[-1].legend(fontsize=8)
    axes[-1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\n{'='*60}")
print(f"Fuzzy Inference Explanation:")
print(f"{'='*60}")
explanation = fis.explain_inference(ctx_sample)
print(f"\nActive Rules (firing strength > 0.01):")
for rule_str, strength in explanation['active_rules'][:5]:  # Show top 5
    print(f"  {rule_str} (strength: {strength:.3f})")
print(f"\nFiring Strengths:")
for label, strength in explanation['firing_strengths'].items():
    if strength > 0:
        print(f"  {label}: {strength:.3f}")

## 7. Summary Statistics

Comprehensive summary of simulation results.

In [ ]:
print(f"{'='*60}")
print(f"Simulation Summary")
print(f"{'='*60}")
print(f"\nScenario: {scenario_name}")
print(f"Policy: Fuzzy Adaptive")
print(f"Duration: {len(df)} slots")

print(f"\nFairness Metrics:")
print(f"  Jain Index:        Mean={df['jain'].mean():.3f}, Std={df['jain'].std():.3f}")
print(f"  Fuzzy Fairness:    Mean={df['fuzzy_fairness'].mean():.3f}, Std={df['fuzzy_fairness'].std():.3f}")
print(f"  α-fairness (α=1):  Mean={df['alpha_1'].mean():.3f}, Std={df['alpha_1'].std():.3f}")
print(f"  Gini Coefficient:  Mean={df['gini'].mean():.3f}, Std={df['gini'].std():.3f}")

print(f"\nThroughput Metrics:")
print(f"  Mean Rate:         {df['mean_rate'].mean()/1e6:.2f} Mbps")
print(f"  Cell Edge Rate:    {df['cell_edge_rate'].mean()/1e6:.2f} Mbps")
print(f"  Min Rate:          {df['mean_rate'].min()/1e6:.2f} Mbps")
print(f"  Max Rate:          {df['mean_rate'].max()/1e6:.2f} Mbps")

print(f"\nOperator Metrics:")
print(f"  Operator Imbalance: Mean={df['operator_imbalance'].mean():.3f}, Std={df['operator_imbalance'].std():.3f}")

print(f"\nAllocation Statistics:")
print(f"  Mean Allocated Users: {df['num_allocated'].mean():.1f} / {df['num_users'].mean():.0f}")
print(f"  Allocation Rate:      {df['num_allocated'].mean() / df['num_users'].mean() * 100:.1f}%")

print(f"\n{'='*60}")